# Stopped Flow Kinetics


## Import Packages

In [ ]:
import numpy as np
from numpy import *
import os,sys,re # Import regex
import pandas as pd # DataFrame analysis

# Plotting
import matplotlib
matplotlib.rcParams.update({'font.size': 20})
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.mplot3d import Axes3D # 3D plots
from matplotlib import cm # Colormaps
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

#Inset figures into plots
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

#Create lines for custom legends
import matplotlib.lines as mlines
from matplotlib.lines import Line2D

from glob import glob

# Insert a progress bar to show the progress of the script
!jupyter nbextension enable --py widgetsnbextension
from tqdm.notebook import tqdm, tnrange, trange

#Scipy Interpolation
import scipy
from scipy.interpolate import splrep, BSpline
from scipy.misc import derivative
from numpy import diff
from scipy.signal import find_peaks

#!pip install lmfit
import lmfit

## 6.1 Import Data

In [ ]:
# Include all 25 temperature. You can use the average, or try to plot the data at each temperature
temps=np.array([[],\
       [],\
       [],\
       [],\
       []])

temps=np.array([25.4,27.5,30,32.5,35])

files=sorted(glob('*csv'))

# Using dictionaries data to import the data and store the temp and run as the key
data={}

for i in range(len(files)):
    data[files[i].split('\\')[-1].split(' ')[1]+"-"+files[i].split('\\')[-1].split(' ')[3].split('.')[0]]=\
    pd.read_csv(files[i],skiprows=[0])

print(data.keys())
keys=list(data.keys())

In [ ]:
# Display and plot the data for the first trial at 25C
plt.plot(data['1-1'].s,data['1-1'].A)

## 6.2 Calculate the derivative

Calculate the derivative and find the respective peaks. This will account for jumps in the data

In [ ]:
X,y=data_sep(data,'1-1')

#Determine A0 as the time (argmin) of the minimum absorbance (peak/valley of reaction)
A0=

#Only consider all data past A0
X2=X[A0:]
y2=y[A0:]

#Consider the one index after A0 to plot the derivative
X3=X2[1:]

# Calculate the derivative numerically
dy_dx=np.diff(y2)/np.diff(X2)

#Plot the derivative
plt.plot(X3,dy_dx)

#Find the peaks of the derivative to identify major changes
peaks, _ = find_peaks(dy_dx)
peaks

In [ ]:
#Filter dy_dx to include major peaks (above a set threshold)
peaks[dy_dx[peaks]>0.05]

In [ ]:
plt.plot(X[A0:A0+6],y[A0:A0+6])
plt.plot(X[A0+13:A0+18],y[A0+13:A0+18])

## 6.2 Create a fitting model

Use an exponential model to fit the segmented data

**How do we think this will relate to**
$$ ln(A_\infty - A) = -k_\text{OBS}t + ln(A_\infty - A_0)$$

In [ ]:
# Create an ex
import lmfit

# Define the exponential model function
def exp_model(x, amp, decay, offset):
    return amp * np.exp(-x / decay) + offset

# Create a Model object from the function
model = lmfit.Model(exp_model)

# Prepare your data (x and y values)
# Assuming combined_X and combined_y are your combined data
combined_X = np.concatenate([X[A0+2:A0+8], X[A0+13:]])
combined_y = np.concatenate([y[A0+2:A0+8], y[A0+13:]])

Xexp = combined_X  
Yexp = combined_y

# Set initial parameter values
params = model.make_params(amp=1, decay=1, offset=0) 

# Perform the fit
result = model.fit(Yexp, params, x=Xexp)

# Plot the data and the fit
plt.plot(Xexp, Yexp, 'o', label='data')
plt.plot(X2,exp_model(X2,result.params['amp'].value,result.params['decay'].value,result.params['offset'].value),'-')

### What does this do?

In [ ]:
print(result.fit_report())

## 6.3 Find the slope

Use the fitted data to get the observed rate ($k_\text{OBS}$)

$$ ln(A_\infty - A) = -k_\text{OBS}t + ln(A_\infty - A_0)$$

In [ ]:
yexp1=exp_model(X2,result.params['amp'].value,result.params['decay'].value,result.params['offset'].value)

logy=np.log(max(yexp1)-yexp1)
logy = logy[np.isfinite(logy)]

plt.plot(X2,np.log(max(yexp1)-yexp1),'o-')

lin=scipy.stats.linregress(X2[logy.index],logy)
lin

### Store the slope and stderr within a 2D list

In the 2D list, the first list will be the slopes while the second list will store the standard error of the slopes

```k25=[[],[]]```


## 6.4 Use functions to search for optimal parameters

Given the choppiness of the data, we can filter out the points based on the derivative to fit the likely exponential data.
We will use functions to screen this effectively.

### Use these functions to test the data points

In [ ]:
#Use these to screen the data points.
def data_sep(data,runs):
    return data[runs].s,data[runs].A


def get_peaks(X,y):
    ''' Plot the derivative to find breaks'''
    X3=X2[1:]
    dy_dx=np.diff(y2)/np.diff(X2)
    plt.plot(X3,dy_dx)

    from scipy.signal import find_peaks
    peaks, _ = find_peaks(dy_dx)
    return peaks

def get_params(x1,x2,x3,x4=-1,X=X,A0=A0, y=y):
    ''' x1, x2, and x3 are parameters past A0 to include in fitting'''
    ''' A splitting between x2 and x3 is allowed for a linear break in between points'''
    
    # Create a Model object from the function
    model = lmfit.Model(exp_model)

    # Prepare your data (x and y values)
    # Assuming combined_X and combined_y are your combined data
    combined_X = np.concatenate([X[A0+x1:A0+x2], X[A0+x3:x4]])
    combined_y = np.concatenate([y[A0+x1:A0+x2], y[A0+x3:x4]])

    plt.plot(X[A0+x1:A0+x2],y[A0+x1:A0+x2], 'ko', label='data1')
    plt.plot(X[A0+x3:x4],y[A0+x3:x4], 'ro', label='data1')
    
    Xexp = combined_X  
    Yexp = combined_y

    # Set initial parameter values
    params = model.make_params(amp=1, decay=1, offset=0) 

    # Perform the fit
    result = model.fit(Yexp, params, x=Xexp)

    # Print the fit results
    print(result.fit_report())

    # Plot the data and the fit
    plt.plot(X2,exp_model(X2,result.params['amp'].value,result.params['decay'].value,result.params['offset'].value),'-')
    plt.show()
    Xexp1, yexp1 = X2, exp_model(X2,result.params['amp'].value,result.params['decay'].value,result.params['offset'].value)
    logy=np.log(max(yexp1)-yexp1)
    logy = logy[np.isfinite(logy)]

    plt.plot(Xexp1,np.log(max(yexp1)-yexp1),'o')
    lin=scipy.stats.linregress(X2[logy.index],logy)
    plt.plot(Xexp1,lin.slope*Xexp1+lin.intercept,'k--',alpha=0.5)
    #Return the fitted parameters from the exponential best fit
    return lin.slope, lin.stderr

### Use these for populating the lists
These remove the plots, which are not necessary after the data points are selected

In [ ]:
def data_sep(data,runs):
    return data[runs].s,data[runs].A

def get_peaks_noplot(X,y):
    ''' Plot the derivative to find breaks'''
    X3=X2[1:]
    dy_dx=np.diff(y2)/np.diff(X2)
    #plt.plot(X3,dy_dx)

    from scipy.signal import find_peaks
    peaks, _ = find_peaks(dy_dx)
    return peaks

def get_params_noplot(x1,x2,x3,x4=-1,X=X,A0=A0, y=y):
    ''' x1, x2, and x3 are parameters past A0 to include in fitting'''
    ''' A splitting between x2 and x3 is allowed for a linear break in between points'''
    
    # Create a Model object from the function
    model = lmfit.Model(exp_model)

    # Prepare your data (x and y values)
    # Assuming combined_X and combined_y are your combined data
    combined_X = np.concatenate([X[A0+x1:A0+x2], X[A0+x3:x4]])
    combined_y = np.concatenate([y[A0+x1:A0+x2], y[A0+x3:x4]])

    Xexp = combined_X  
    Yexp = combined_y

    # Set initial parameter values
    params = model.make_params(amp=1, decay=1, offset=0) 

    # Perform the fit
    result = model.fit(Yexp, params, x=Xexp)

    Xexp1, yexp1 = X2, exp_model(X2,result.params['amp'].value,result.params['decay'].value,result.params['offset'].value)
    logy=np.log(max(yexp1)-yexp1)
    logy = logy[np.isfinite(logy)]

    lin=scipy.stats.linregress(X2[logy.index],logy)
    return lin.slope, lin.stderr

#### For example

In [ ]:
X,y=data_sep(data,'2-5')
A0=np.argmin(y)
X2=X[A0:]
y2=y[A0:]

plt.plot(X,y)
plt.show()

peaks = get_peaks(X,y)
print(peaks+1)
plt.show()

k, ke = get_params(2,5,14,-1,X,A0,y)
print(k)

In [ ]:
k25=[[],[]]

X,y=data_sep(data,'1-1')
A0=np.argmin(y)
X2=X[A0:]
y2=y[A0:]

k, ke = get_params_noplot(2,8,13,-1,X,A0,y)

k25[0].append(k)
k25[1].append(ke)

#-------------------------------
X,y=data_sep(data,'1-2')
A0=np.argmin(y)
X2=X[A0:]
y2=y[A0:]

k, ke = get_params_noplot(2,4,6,-1,X,A0,y)

k25[0].append(k)
k25[1].append(ke)

#-------------------------------
X,y=data_sep(data,'1-3')
A0=np.argmin(y)
X2=X[A0:]
y2=y[A0:]

k, ke = get_params_noplot(1,3,3,A0+12,X,A0,y)

k25[0].append(k)
k25[1].append(ke)

#-------------------------------
X,y=data_sep(data,'1-4')
A0=np.argmin(y)
X2=X[A0:]
y2=y[A0:]

k, ke = get_params_noplot(4,7,7,-1,X,A0,y)

k25[0].append(k)
k25[1].append(ke)

#-------------------------------
X,y=data_sep(data,'1-5')
A0=np.argmin(y)
X2=X[A0:]
y2=y[A0:]

k, ke = get_params_noplot(2,7,7,A0+16,X,A0,y)

k25[0].append(k)
k25[1].append(ke)


k25=np.array(k25)

In [ ]:
# Repeat this for each set of temperatures

## 6.5 Calculate $E_a$

Plot $ln(k_\text{OBS})$ vs. $\dfrac{1}{T}$ to calculate $E_a$ via
$$ln(k_{OBS}) = -\dfrac{E_A}{R}*\dfrac{1}{T} + ln A$$

